<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/qiskit_kernel_sweep_updated_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install qiskit scikit-learn numpy matplotlib

In [ ]:
#!/usr/bin/env python3
"""
qiskit_kernel_sweep_updated.py

2-qubit ZZ-feature-map fidelity-kernel sweep under class imbalance.

- Generates 2D toy data at three imbalance levels
- Builds ZZFeatureMap using zz_feature_map API
- Computes fidelity^2 kernel, centers it, measures alignment
- Evaluates SVM CV accuracy with n_splits=min(class_size,5)
"""

import warnings
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.svm import SVC
from qiskit.circuit.library import zz_feature_map
from qiskit.quantum_info import Statevector, state_fidelity

# Suppress DeprecationWarning from Qiskit
warnings.filterwarnings("ignore", category=DeprecationWarning)

def generate_data(n_samples, imbalance, random_state=42):
    if imbalance >= 1.0:
        weights = [0.5, 0.5]
    else:
        weights = [imbalance, 1.0 - imbalance]
    X, y = make_classification(n_samples=n_samples,
                               n_features=2,
                               n_redundant=0,
                               n_clusters_per_class=1,
                               weights=weights,
                               class_sep=2.0,
                               random_state=random_state)
    return X, y

def build_feature_map(depth):
    # Use the recommended zz_feature_map function
    return zz_feature_map(feature_dimension=2,
                          reps=depth,
                          entanglement='circular',
                          insert_barriers=False)

def compute_fidelity_kernel(feature_map, X):
    n = len(X)
    # Precompute statevectors
    psi = [Statevector.from_instruction(feature_map.assign_parameters(x)) for x in X]

    # Fidelity-squared Gram matrix
    K = np.zeros((n, n), float)
    for i in range(n):
        for j in range(i, n):
            f = state_fidelity(psi[i], psi[j])**2
            K[i, j] = K[j, i] = f
    return K

def center_kernel(K):
    n = K.shape[0]
    one_n = np.ones((n, n)) / n
    return K - one_n @ K - K @ one_n + one_n @ K @ one_n

def kernel_target_alignment(Kc, y):
    yy = np.outer(y, y)
    num = np.trace(Kc @ yy)
    den = np.linalg.norm(Kc, 'fro') * np.linalg.norm(yy, 'fro')
    return num / den

def svm_cv_scores(K, y, max_splits=5):
    # Determine n_splits ≤ min(counts)
    classes, counts = np.unique(y, return_counts=True)
    min_count = counts.min()
    n_splits = min(max_splits, min_count)
    if len(classes) < 2 or n_splits < 2:
        return np.array([])
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)
    svc = SVC(kernel='precomputed')
    return cross_val_score(svc, K, y, cv=cv)

def main():
    np.random.seed(0)
    n_samples = 10
    imbalance_levels = [1.0, 0.75, 0.50]
    depths = [1, 3, 5]

    for imb in imbalance_levels:
        X, y = generate_data(n_samples, imb, random_state=1)

        # Print header with baseline CNOT-depth-3 & “QNN” placeholder
        K3 = compute_fidelity_kernel(build_feature_map(3), X)
        scores3 = svm_cv_scores(K3, y)
        mean3, std3 = scores3.mean(), scores3.std()
        print(f"Imbalance {int(imb*100)}%: CNOT={mean3:.3f}±{std3:.3f}, QNN={mean3:.3f}±{std3:.3f}")

        for d in depths:
            fmap = build_feature_map(d)
            K = compute_fidelity_kernel(fmap, X)
            Kc = center_kernel(K)
            align = kernel_target_alignment(Kc, y)
            scores = svm_cv_scores(K, y)
            if scores.size > 0:
                m, s = scores.mean(), scores.std()
                print(f"  depth={d} | align={align:.3f} | SVM={m:.3f}±{s:.3f}")
            else:
                print(f"  depth={d} | align={align:.3f} | SVM= n/a")
        print()

if __name__ == "__main__":
    main()